Baseado no artigo: https://machinelearningmastery.com/elastic-net-regression-in-python/

**O Elastic Net é um modelo de penalização linear que utiliza as abordagens L1 e L2;**

---

*One popular penalty is to penalize a model based on the sum of the squared coefficient values. This is called an L2 penalty. An L2 penalty minimizes the size of all coefficients, although it prevents any coefficients from being removed from the model.*

- l2_penalty = sum j=0 to p beta_j^2

*Another popular penalty is to penalize a model based on the sum of the absolute coefficient values. This is called the L1 penalty. An L1 penalty minimizes the size of all coefficients and allows some coefficients to be minimized to the value zero, which removes the predictor from the model.*

- l1_penalty = sum j=0 to p abs(beta_j)

---

**Hiperparâmetros:**

*A hyperparameter “alpha” is provided to assign how much weight is given to each of the L1 and L2 penalties. Alpha is a value between 0 and 1 and is used to weight the contribution of the L1 penalty and one minus the alpha value is used to weight the L2 penalty.*

- elastic_net_penalty = (alpha * l1_penalty) + ((1 – alpha) * l2_penalty)

*For example, an alpha of 0.5 would provide a 50 percent contribution of each penalty to the loss function. An alpha value of 0 gives all weight to the L2 penalty and a value of 1 gives all weight to the L1 penalty.*

*Another hyperparameter is provided called “lambda” that controls the weighting of the sum of both penalties to the loss function. A default value of 1.0 is used to use the fully weighted penalty; a value of 0 excludes the penalty. Very small values of lambada, such as 1e-3 or smaller, are common.*

- elastic_net_loss = loss + (lambda * elastic_net_penalty)




## Elastic Net em regressão

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
df = pd.read_csv('/content/drive/MyDrive/myColab/datasets/regressao.csv')
df.head()

,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,target
0,-0.97127,-0.56128,1.20414,2.25145,-0.25802,0.06246,-1.78623,-0.27690,-0.22255,0.20963,-0.062202
1,0.89456,-0.94099,-0.19972,-0.08354,-0.96865,0.62801,0.91543,0.20413,-0.13587,0.96820,0.112596
2,-0.98777,-0.55565,-0.94141,-1.86277,0.38659,-0.02523,-1.26714,1.07328,-0.28350,0.34430,0.167633
3,0.03972,-0.99258,0.63481,0.01554,-0.39203,-0.72647,0.46098,0.79395,0.22686,0.25065,-0.844727
4,-1.66022,-0.06734,-0.72252,-1.32035,0.06715,2.39937,-1.30566,-0.65222,0.36628,0.31644,2.221942


In [2]:
# Training data
x, y = df.iloc[:, 0:-1], df.iloc[:, -1]

*Using scikit-learn confusingly, the alpha hyperparameter can be set via the “l1_ratio” argument that controls the contribution of the L1 and L2 penalties and the lambda hyperparameter can be set via the “alpha” argument that controls the contribution of the sum of both penalties to the loss function.*

In [3]:
# Using elastic net pure
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import ElasticNet

# defaults to alpha (lambda) and l1_ratio (alpha)
model = ElasticNet(alpha=1.0, l1_ratio=0.5)

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_val_score(model, x, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores = np.absolute(scores)
print(f'MAE mean: {np.mean(scores)} - STD mean {np.std(scores)}')

MAE mean: 0.6295942848788132 - STD mean 0.02511000542566693


In [4]:
# Using grid search to determine alpha (lambda) and l1_ratio (alpha)
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet

grid = dict()
grid['alpha'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0]
grid['l1_ratio'] = np.arange(0, 1, 0.01)

model = ElasticNet()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
results = search.fit(x, y)
print(f'Best MAE: {results.best_score_} - Best params {results.best_params_}')

Best MAE: -0.00041167052274532353 - Best params {'alpha': 1e-05, 'l1_ratio': 0.99}


In [5]:
# Using grid from elastic net
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import ElasticNetCV

alphas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0]
ratios = np.arange(0, 1, 0.01)

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
model = ElasticNetCV(l1_ratio=ratios, alphas=alphas, cv=cv, n_jobs=-1)
model.fit(x, y)
print(f'Mean MAE: {np.sqrt(np.mean(model.mse_path_))} - Best Alpha {model.alpha_} - Best L1_ratio {model.l1_ratio_}')

Mean MAE: 0.5414544289298434 - Best Alpha 1e-05 - Best L1_ratio 0.99


## Elastic Net em classificação
Para uso em problemas de classificação utilizamos o algoritmo de Regressão Logística com uma penalidade baseada em Elastic Net.

In [6]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
df = pd.read_csv('/content/drive/MyDrive/myColab/datasets/iris.csv')
df.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [7]:
cls = {
    'Setosa': 0,
    'Versicolor': 1,
    'Virginica': 2}

df['variety'] = df.apply(lambda x: cls[x.variety], axis=1)
df.variety.unique()

array([0, 1, 2])

In [8]:
# Training data
x, y = df.iloc[:, 0:-1], df.iloc[:, -1]

In [9]:
# Using elastic net pure
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LogisticRegression

# defaults to alpha (lambda) and l1_ratio (alpha)
model = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5)

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_val_score(model, x, y, scoring='accuracy', cv=cv, n_jobs=-1)
scores = np.absolute(scores)
print(f'Accuracy mean: {np.mean(scores)} - STD mean {np.std(scores)}')

MAE mean: 0.9755555555555556 - STD mean 0.043829073162924476


In [10]:
# Using grid search to determine alpha (lambda) and l1_ratio (alpha)
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

grid = dict()
grid['penalty'] = ['elasticnet']
# Solvers que funcionam com multiclass
grid['solver'] = ['saga', 'sag', 'newton-cg']
grid['l1_ratio'] = np.arange(0, 1, 0.01)

model = LogisticRegression()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
search = GridSearchCV(model, grid, scoring='accuracy', cv=cv, n_jobs=-1)
results = search.fit(x, y)
print(f'Best Accuracy: {results.best_score_} - Best params {results.best_params_}')

Best MAE: 0.9755555555555556 - Best params {'l1_ratio': 0.47000000000000003, 'penalty': 'elasticnet', 'solver': 'saga'}


In [16]:
# Using grid from logistic regression
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LogisticRegressionCV

penalty = 'elasticnet'
solvers = 'saga'
scoring = 'accuracy'
multi_class = 'multinomial'
ratios = np.arange(0, 1, 0.01)

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
model = LogisticRegressionCV(
    penalty=penalty,
    solver=solvers,
    l1_ratios=ratios,
    scoring=scoring,
    multi_class = multi_class,
    cv=cv,
    n_jobs=-1)
model.fit(x, y)
print(f'Best Accuracy: {np.mean(model.C_)} - Best L1_ratio {model.l1_ratio_}')

Best Accuracy: 0.3593813663804626 - Best L1_ratio [0.94 0.94 0.94]
